---
# Compiled by Bumsik Kim (Virginia Tech, kismub@gmail.com)

## Feb 1, 2019

## Code for I-95 Network V1
### Use filepath as itself
---

# Import

In [1]:
import win32com.client as com
import os
import pandas as pd
import time


# import win32com
# print(win32com.__gen_path__
# GIT


# Initiate VISSIM

In [6]:
## First, initiate VISSIM program by following code
## Connecting the COM Server => Open a new Vissim Window:
#Vissim = win32com.client.dynamic.Dispatch("Vissim.Vissim-64.10") # Vissim 10 - 64 bit


###############################################################################
## IF IT DOESNT WORK, DELETE gen_py at C:\Users\CEE Admin\AppData\Local\Temp ##
###############################################################################
#Vissim = win32com.client.dynamic.Dispatch("Vissim.Vissim-64.10") # Vissim 10 - 64 bit 
Vissim = com.gencache.EnsureDispatch("Vissim.Vissim-64.10") # Vissim 10 - 64 bit 



#Vissim = com.Dispatch("Vissim.Vissim-64.10")
## Load working directory
Path_of_COM_Basic_Commands_network = os.getcwd()
Path_of_COM_Basic_Commands_network = os.path.abspath(os.path.join(Path_of_COM_Basic_Commands_network, os.path.pardir))
print (Path_of_COM_Basic_Commands_network)
#Path_of_COM_Basic_Commands_network = 'C:\\Users\\CEE Admin\\Documents\\VISSIM\\I95_VDOT_Base\\PM_Working\\'

File = '200824_i-95_pm_CAV_WO_Sig'

## Load a Vissim Network:
Filename                = os.path.join(Path_of_COM_Basic_Commands_network, File + '.inpx')
flag_read_additionally  = False # you can read network(elements) additionally, in this case set "flag_read_additionally" to true
Vissim.LoadNet(Filename, flag_read_additionally)
## Load a Layout:
Filename = os.path.join(Path_of_COM_Basic_Commands_network, File + '.layx')
Vissim.LoadLayout(Filename)

# Functions

In [3]:
def simulation(Randomseed, SimEndTime):
    # Delete all previous simulation runs first:
    for simRun in Vissim.Net.SimulationRuns:
        Vissim.Net.SimulationRuns.RemoveSimulationRun(simRun)

    # Activate QuickMode:
    Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode", 1)
    Vissim.SuspendUpdateGUI();  # stop updating of the complete Vissim workspace (network editor, list, chart and signal time table windows)
    # Alternatively, load a layout (*.layx) where dynamic elements (vehicles and pedestrians) are not visible:
    # Vissim.LoadLayout(os.path.join(Path_of_COM_Basic_Commands_network, 'COM Basic Commands - Hide vehicles.layx')) # loading a layout where vehicles are not displayed
    End_of_simulation = SimEndTime
    Vissim.Simulation.SetAttValue('SimPeriod', End_of_simulation)
    Sim_break_at = 0  # simulation second [s] => 0 means no break!
    Vissim.Simulation.SetAttValue('SimBreakAt', Sim_break_at)
    # Set maximum speed:
    Vissim.Simulation.SetAttValue('UseMaxSimSpeed', True)

    # Change randomseed
    Vissim.Simulation.SetAttValue('RandSeed', Randomseed)  # Note: RandSeed 0 is not allowed
    Vissim.Simulation.RunContinuous()

    Vissim.ResumeUpdateGUI(True);  # allow updating of the complete Vissim workspace (network editor, list, chart and signal time table windows)
    Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode", 0)  # deactivate QuickMode
    # Vissim.LoadLayout(os.path.join(Path_of_COM_Basic_Commands_network, 'COM Basic Commands.layx')) # loading a layout to display vehicles again

    # List of all Simulation runs:
    Attributes = ['Timestamp', 'RandSeed', 'SimEnd']
    number_of_runs = Vissim.Net.SimulationRuns.Count
    List_Sim_Runs = Vissim.Net.SimulationRuns.GetMultipleAttributes(Attributes)
    for cnt_S in range(number_of_runs):
        print('Start at %s | with Randomseed %d | Simulation ends at %d' % (List_Sim_Runs[cnt_S][0], List_Sim_Runs[cnt_S][1], List_Sim_Runs[cnt_S][2]))

def get_traveltime(TTM, TimeInterval):
    # Get the results of Vehicle Travel Time Measurements:
    MOE_TTM = [] ## Save output and return as a list
        
    if TTM==0:
        for i in Vissim.Net.VehicleTravelTimeMeasurements: ## Loop, all vehicle travel time measurements
            for j in TimeInterval:
                if j==0:
                    ID = 'NO'
                    Name = 'Name'
                    Time = 'TravTm(Avg,Avg,All)'
                    Vehs = 'Vehs  (Avg,Avg,All)'
                else:
                    ID = 'NO'
                    Name = 'Name'
                    Time = 'TravTm(Avg,'+str(j)+',All)'
                    Vehs = 'Vehs  (Avg,'+str(j)+',All)'

                # Average of all simulations (1. input = Avg)
                # of the average of all time intervals  (2. input = Avg)
                # of all vehicle classes (3. input = All)
                Veh_TTM_ID = i.AttValue(ID) ## TTM ID
                Veh_TTM_Name = i.AttValue(Name) ## TTM Name
                Veh_TTM_Time = i.AttValue(Time) ## Travel time
                Veh_TTM_Vehs = i.AttValue(Vehs)
                MOE_TTM.append([Veh_TTM_ID, Veh_TTM_Name, j, Veh_TTM_Time, Veh_TTM_Vehs]) ## ID, TTM_Name, TimeInterval, Travel Time, Number of Vehicles
    else:
        Veh_TTM_ID = Vissim.Net.VehicleTravelTimeMeasurements.ItemByKey(TTM).AttValue('No') ## TTM ID
        Veh_TTM_Name = Vissim.Net.VehicleTravelTimeMeasurements.ItemByKey(TTM).AttValue('Name') ## TTM ID
        Veh_TTM_Time = Vissim.Net.VehicleTravelTimeMeasurements.ItemByKey(TTM).AttValue('TravTm(Avg,Avg,All)')
        Veh_TTM_Vehs = Vissim.Net.VehicleTravelTimeMeasurements.ItemByKey(TTM).AttValue('Vehs  (Avg,Avg,All)')
        MOE_TTM.append([Veh_TTM_ID, Veh_TTM_Name, Veh_TTM_Time, Veh_TTM_Vehs])
    
    # Return list, [[ID, Name, Travel time, Number of vehicles]]
    return MOE_TTM


def get_data_DCM(DCM, TimeInterval):
    # Get data from Data Collection Measurement
    # Must define Data Collection Measurement from VISSIM first!!!!!
    # Evaluation -> Measurement Definition -> DCM
    
    MOE_DCM = [] ## Save output and return as a list
    
    if DCM==0:
        for DC_measurement in Vissim.Net.DataCollectionMeasurements: ## Loop, all vehicle travel time measurements
            for j in TimeInterval:
                if j==0:
                    No = 'Concatenate:DataCollectionPoints\\No'
                    Vehs = 'Vehs        (Avg,Avg,All)'
                    Spd = 'Speed       (Avg,Avg,All)'
                    Acc = 'Acceleration(Avg,Avg,All)'
                    Len = 'Length      (Avg,Avg,All)'
                    Dly = 'QueueDelay  (Avg,Avg,All)'
                else:
                    No = 'Concatenate:DataCollectionPoints\\No'
                    Vehs = 'Vehs        (Avg,'+str(j)+',All)'
                    Spd = 'Speed       (Avg,'+str(j)+',All)'
                    Acc = 'Acceleration(Avg,'+str(j)+',All)'
                    Len = 'Length      (Avg,'+str(j)+',All)'
                    Dly = 'QueueDelay  (Avg,'+str(j)+',All)'
                    
                DCM_No          = DC_measurement.AttValue(No) # Number of data collection points
                No_Veh          = DC_measurement.AttValue(Vehs) # number of vehicles (Simulation average, 1st time interval, all vehicle classes)
                Speed           = DC_measurement.AttValue(Spd) # Speed of vehicles
                Acceleration    = DC_measurement.AttValue(Acc) # Acceleration of vehicles
                Length          = DC_measurement.AttValue(Len) # Length of vehicles
                Delay           = DC_measurement.AttValue(Dly) # Queue Delay

                MOE_DCM_TMP = []
                MOE_DCM_TMP = [DCM_No, j, No_Veh, Speed, Acceleration, Length, Delay] ## DCM NO, TimeInterval, #ofVeh, Spd, Acc, Len, Delay
                MOE_DCM.append(MOE_DCM_TMP)
            
        
    else:
        DC_measurement=Vissim.Net.DataCollectionMeasurements.ItemByKey(DCM)
        DCM_No          = DC_measurement.AttValue('Concatenate:DataCollectionPoints\\No') # Number of data collection points
        No_Veh          = DC_measurement.AttValue('Vehs        (Avg,1,All)') # number of vehicles (Simulation average, 1st time interval, all vehicle classes)
        Speed           = DC_measurement.AttValue('Speed       (Avg,1,All)') # Speed of vehicles
        Acceleration    = DC_measurement.AttValue('Acceleration(Avg,1,All)') # Acceleration of vehicles
        Length          = DC_measurement.AttValue('Length      (Avg,1,All)') # Length of vehicles
        Delay           = DC_measurement.AttValue('QueueDelay  (Avg,1,All)') # Queue Delay
        MOE_DCM_TMP = []
        MOE_DCM_TMP = [DCM_No, No_Veh, Speed, Acceleration, Length, Delay]
        MOE_DCM.append(MOE_DCM_TMP)
        
    return MOE_DCM
    

def get_queue(QC, TimeInterval):
    MOE_QC = [] ## Save output and return as a list
    if QC==0:
        for i in Vissim.Net.QueueCounters:
            for j in TimeInterval:
                if j==0:
                    No = 'No'
                    Q = 'QLenMax(Avg, Avg)'
                else:
                    No = 'No'
                    Q = 'QLenMax(Avg, '+str(j)+')'
                    
                QC_Num = i.AttValue(No)
                MaxQ = i.AttValue(Q)
                MOE_QC.append([QC_Num, j, MaxQ])
    else:
        QC_Num = Vissim.Net.QueueCounters.ItemByKey(QC).AttValue('No')
        MaxQ = Vissim.Net.QueueCounters.ItemByKey(QC).AttValue('QLenMax(Avg, Avg)')
        MOE_QC.append(QC_Num, MaxQ)
    return MOE_QC ## Return [Queue counter number, Max Queue length]
    
def get_density(LinkNum, TimeInterval):
    MOE_Density = []
    if LinkNum==0:
        for i in Vissim.Net.Links:
            for j in TimeInterval:
                if j==0:
                    No = 'No'
                    Seg = 'LinkEvalSegment'
                    Density = 'Concatenate:LinkEvalSegs\Density(Avg, Avg, All)'
                    Speed = 'Concatenate:LinkEvalSegs\Speed(Avg, Avg, All)'
                    Volume = 'Concatenate:LinkEvalSegs\Volume(Avg, Avg, All)'
                else:
                    No = 'No'
                    Seg = 'LinkEvalSegment'
                    Density = 'Concatenate:LinkEvalSegs\Density(Avg, '+str(j)+', All)'
                    Speed = 'Concatenate:LinkEvalSegs\Speed(Avg, '+str(j)+', All)'
                    Volume = 'Concatenate:LinkEvalSegs\Volume(Avg, '+str(j)+', All)'
            
                Link_Num = i.AttValue(No)
                Link_Density = i.AttValue(Density)
                Link_Density = Link_Density.split(',') ## Used for multiple segment 
                Link_Speed = i.AttValue(Speed)
                Link_Speed = Link_Speed.split(',') ## Used for multiple segment 
                Link_Volume = i.AttValue(Volume)
                Link_Volume = Link_Volume.split(',') ## Used for multiple segment 
                
                Seg_length = list(range(0,len(Link_Speed))) 
                
                for k in Seg_length:
                    Segment_ID = k
                    Link_Density_Seg = Link_Density[k]
                    Link_Speed_Seg = Link_Speed[k]
                    Link_Volume_Seg = Link_Volume[k]
                    
                    MOE_Density.append([Link_Num, j, Segment_ID, Link_Density_Seg, Link_Speed_Seg, Link_Volume_Seg])
    else:
        Link_Num = Vissim.Net.Links.ItemByKey(LinkNum).AttValue('No')
        Link_Density=Vissim.Net.Links.ItemByKey(LinkNum).AttValue('Concatenate:LinkEvalSegs\Density(Avg, Avg, All)')
        Link_Density = Link_Density.split(',')
        Link_Speed=Vissim.Net.Links.ItemByKey(LinkNum).AttValue('Concatenate:LinkEvalSegs\Speed(Avg, Avg, All)')
        Link_Speed = Link_Speed.split(',')
        Link_Volume=Vissim.Net.Links.ItemByKey(LinkNum).AttValue('Concatenate:LinkEvalSegs\Volume(Avg, Avg, All)')
        Link_Volume = Link_Volume.split(',')
        Seg_length = list(range(0,len(Link_Speed)))
        for k in Seg_length:
            Segment_ID = k
            Link_Density_Seg = Link_Density[k]
            Link_Speed_Seg = Link_Speed[k]
            Link_Volume_Seg = Link_Volume[k]
            MOE_Density.append([Link_Num, j, Segment_ID, Link_Density_Seg, Link_Speed_Seg, Link_Volume_Seg])
    return MOE_Density

def get_delay(TTM, TimeInterval):
    # Get the delay from Vehicle Travel Time Measurements:
    MOE_DELAY = [] ## Save output and return as a list

    if TTM==0:
        for i in Vissim.Net.DelayMeasurements: ## Loop, all vehicle travel time measurements
            for j in TimeInterval:
                if j==0:
                    No = 'No'
                    VehDelay = 'VehDelay(Avg,Avg,All)'
                    Vehs = 'Vehs(Avg,Avg,All)'
                else:
                    No = 'No'
                    VehDelay = 'VehDelay(Avg,'+str(j)+',All)'
                    Vehs = 'Vehs(Avg,'+str(j)+',All)'
                # Average of all simulations (1. input = Avg)
                # of the average of all time intervals  (2. input = Avg)
                # of all vehicle classes (3. input = All)
                Veh_DELAY_ID = i.AttValue(No) ## TTM ID
                Veh_DELAY_VehDelay = i.AttValue(VehDelay) ## Vehicle delay (seconds)
                Veh_DELAY_Vehs = i.AttValue(Vehs) ## Number of vehicles
                MOE_DELAY.append([Veh_DELAY_ID, j, Veh_DELAY_VehDelay, Veh_DELAY_Vehs])
    else:
        Veh_DELAY_ID = Vissim.Net.DelayMeasurements.ItemByKey(TTM).AttValue('No') ## TTM ID
        Veh_DELAY_VehDelay = Vissim.Net.DelayMeasurements.ItemByKey(TTM).AttValue('VehDelay(Avg,Avg,All)') ## Vehicle delay (seconds)
        Veh_DELAY_Vehs = Vissim.Net.DelayMeasurements.ItemByKey(TTM).AttValue('Vehs(Avg,Avg,All)') ## Number of vehicles
        MOE_DELAY.append([Veh_DELAY_ID, Veh_DELAY_VehDelay, Veh_DELAY_Vehs])
        
    return MOE_DELAY



def saving_file():
    Filename = os.path.join(Path_of_COM_Basic_Commands_network, 'AV_Test_output.inpx')
    Vissim.SaveNetAs(Filename)
    Filename = os.path.join(Path_of_COM_Basic_Commands_network, 'AV_Test_output.layx')
    Vissim.SaveLayout(Filename)

    # ## ========================================================================
    # # End Vissim
    # #==========================================================================
    # Vissim = None

    # # To stop the simulation:
    # Vissim.Simulation.Stop()

def get_vehicle_input(VI_number):
    if VI_number==0:
        Volume_of_Input_Total = Vissim.Net.VehicleInputs.GetMultiAttValues('Volume(1)')
    else:
        Volume_of_Input_Total = Vissim.Net.VehicleInputs.ItemByKey(VI_number).AttValue('Volume(1)')
        ## Time interval, (1) means first time interval
    return Volume_of_Input_Total
    
def set_vehicle_input(VI_number, Volume):
    # Set vehicle input:
    # VI = Vehicle Input number, Volume = new volume, Composition = new vehicle composition
    
    if VI_number==0:
        Num_of_Input_Total = Vissim.Net.VehicleInputs.GetMultiAttValues('No') 
        print (Num_of_Input_Total)
        for i in range(len(Num_of_Input_Total)):
            Num_of_Input=Num_of_Input_Total[i][1]
            Vissim.Net.VehicleInputs.ItemByKey(Num_of_Input).SetAttValue('Volume(1)', Volume)
    else:
        Vissim.Net.VehicleInputs.ItemByKey(VI_number).SetAttValue('Volume(1)', Volume)
    
    # 'Volume(1)' means the first defined time interval
    # Hint: The Volumes of following intervals Volume(i) i = 2...n can only be
    # edited, if continuous is deactivated: (otherwise error: "AttValue failed: Object 2: Attribute Volume (300) is no subject to changes.")
    #Vissim.Net.VehicleInputs.ItemByKey(VI_number).SetAttValue('Cont(2)', False)
    #Vissim.Net.VehicleInputs.ItemByKey(VI_number).SetAttValue('Volume(2)', 400)


## Change Market Penetration by using pre-defined vehicle composition in VISSIM
## Each Vehicle composition level represents each base scenario
def set_vehicle_composition(VI_number, TimeInterval_number, Composition):
    # VI_number = Vehicle Input number, Composition = vehicle composition number
    # Vehicle composition number = integer, starting 1
    # If vehicle compoisition number is 0, it means all vehicle input use same composition number
    if VI_number==0:
        for i in TimeInterval_number:
            VehComp = 'VehComp('+str(i)+')'
            Vissim.Net.VehicleInputs.SetAllAttValues(VehComp, Composition)
    else:
        Vissim.Net.VehicleInputs.ItemByKey(VI_number).SetAttValue('VehComp(1)', Composition)




    
def gen_vehicle_composition(vehiclecomp_list, SpdDist_PV, SpdDist_LT, SpdDist_HT): ## vehiclecomp_list is a list (scenario number)
    ## Before start, the vehicle type should be pre-defined in VISSIM
    ## 800: PV legacy, 810: PV AV, 820: PV CAV, 900: HV legaccy, 910: HV AV, 920: HV CAV
    for vehiclecomp_num in vehiclecomp_list:
        ## Check vehicle composition number is in existing vehicle composition
        ## Create empty 'vehicle composition exist' list to store the ID of vehicle composition
        VehComp_exist = []
        for i in Vissim.Net.VehicleCompositions:
            VehComp_exist.append(i.AttValue('No'))
        ## If the vehicle composition number is exist in the existing VC, remove and create again
        ## If the vehicle composition is not exist, just create one
        
        
        if vehiclecomp_num in VehComp_exist:
            Vissim.Net.VehicleCompositions.RemoveVehicleComposition(Vissim.Net.VehicleCompositions.ItemByKey(vehiclecomp_num)) ## NOOOOO
            #Vissim.Net.VehicleCompositions.AddVehicleComposition(vehiclecomp_num,[])
            Vissim.Net.VehicleCompositions.AddVehicleComposition(vehiclecomp_num, [Vissim.Net.VehicleTypes.ItemByKey(800), Vissim.Net.DesSpeedDistributions.ItemByKey(SpdDist_PV)]) # generate vehicle composition relative flow (vehicle type 800, speed dist 25)
            print ('Remove %d and create again' %(vehiclecomp_num))
   
        else:
            Vissim.Net.VehicleCompositions.AddVehicleComposition(vehiclecomp_num, [Vissim.Net.VehicleTypes.ItemByKey(800), Vissim.Net.DesSpeedDistributions.ItemByKey(SpdDist_PV)]) # generate vehicle composition relative flow (vehicle type 800, speed dist 25)
            print ('Create %d' %(vehiclecomp_num))
        ## Generate relative flow
        Vissim.Net.VehicleCompositions.ItemByKey(vehiclecomp_num).VehCompRelFlows.AddVehicleCompositionRelativeFlow(Vissim.Net.VehicleTypes.ItemByKey(810), Vissim.Net.DesSpeedDistributions.ItemByKey(SpdDist_PV)) # IVehicleType* VehType, IDesSpeedDistribution* DesSpeedDistr    
        Vissim.Net.VehicleCompositions.ItemByKey(vehiclecomp_num).VehCompRelFlows.AddVehicleCompositionRelativeFlow(Vissim.Net.VehicleTypes.ItemByKey(820), Vissim.Net.DesSpeedDistributions.ItemByKey(SpdDist_PV)) # IVehicleType* VehType, IDesSpeedDistribution* DesSpeedDistr    
        Vissim.Net.VehicleCompositions.ItemByKey(vehiclecomp_num).VehCompRelFlows.AddVehicleCompositionRelativeFlow(Vissim.Net.VehicleTypes.ItemByKey(900), Vissim.Net.DesSpeedDistributions.ItemByKey(SpdDist_LT)) # IVehicleType* VehType, IDesSpeedDistribution* DesSpeedDistr    
        Vissim.Net.VehicleCompositions.ItemByKey(vehiclecomp_num).VehCompRelFlows.AddVehicleCompositionRelativeFlow(Vissim.Net.VehicleTypes.ItemByKey(910), Vissim.Net.DesSpeedDistributions.ItemByKey(SpdDist_LT)) # IVehicleType* VehType, IDesSpeedDistribution* DesSpeedDistr    
        Vissim.Net.VehicleCompositions.ItemByKey(vehiclecomp_num).VehCompRelFlows.AddVehicleCompositionRelativeFlow(Vissim.Net.VehicleTypes.ItemByKey(920), Vissim.Net.DesSpeedDistributions.ItemByKey(SpdDist_LT)) # IVehicleType* VehType, IDesSpeedDistribution* DesSpeedDistr    
        Vissim.Net.VehicleCompositions.ItemByKey(vehiclecomp_num).VehCompRelFlows.AddVehicleCompositionRelativeFlow(Vissim.Net.VehicleTypes.ItemByKey(950), Vissim.Net.DesSpeedDistributions.ItemByKey(SpdDist_HT)) # IVehicleType* VehType, IDesSpeedDistribution* DesSpeedDistr    
        Vissim.Net.VehicleCompositions.ItemByKey(vehiclecomp_num).VehCompRelFlows.AddVehicleCompositionRelativeFlow(Vissim.Net.VehicleTypes.ItemByKey(960), Vissim.Net.DesSpeedDistributions.ItemByKey(SpdDist_HT)) # IVehicleType* VehType, IDesSpeedDistribution* DesSpeedDistr    
        Vissim.Net.VehicleCompositions.ItemByKey(vehiclecomp_num).VehCompRelFlows.AddVehicleCompositionRelativeFlow(Vissim.Net.VehicleTypes.ItemByKey(970), Vissim.Net.DesSpeedDistributions.ItemByKey(SpdDist_HT)) # IVehicleType* VehType, IDesSpeedDistribution* DesSpeedDistr    
        
        
def set_vehicle_composition_scenario(MP_Scenario):
    Number_of_MP = MP_Scenario.shape[0]
    for i in range(Number_of_MP):
        Scenario_Num = int(MP_Scenario.iloc[i][0])
        Pleg = MP_Scenario.iloc[i][1]
        Pav = MP_Scenario.iloc[i][2]
        Pcav = MP_Scenario.iloc[i][3]
        LTleg = MP_Scenario.iloc[i][4]
        LTav = MP_Scenario.iloc[i][5]
        LTcav = MP_Scenario.iloc[i][6]
        HTleg = MP_Scenario.iloc[i][7]
        HTav = MP_Scenario.iloc[i][8]
        HTcav = MP_Scenario.iloc[i][9]
        AttName = ("VehType", "RelFlow")
        Attributes = ((800,Pleg),(810,Pav),(820,Pcav),(900,LTleg),(910,LTav),(920,LTcav),(950,HTleg),(960,HTav),(970,HTcav))
        Vissim.Net.VehicleCompositions.ItemByKey(Scenario_Num).VehCompRelFlows.SetMultipleAttributes(AttName, Attributes)
        Comp_name = ("Pleg" + str(Pleg) + " PAV" + str(Pav) + " PCAV" + str(Pcav) + " LTleg"+str(LTleg) + " LTAV" + str(LTav) + " LTCAV" + str(LTcav) + " HTleg"+str(HTleg) + " HTAV" + str(HTav) + " HTCAV" + str(HTcav))
        Vissim.Net.VehicleCompositions.ItemByKey(Scenario_Num).SetAttValue('Name', Comp_name)


def get_vehicle_composition_scenario(File):
    MP_Scenario = pd.read_csv(File)
    return (MP_Scenario)


def set_linkEvalSeg_99999 (LinkNum): ## Set link evaluation segment as 99999m for getting a whole link density
    if LinkNum==0:
        for i in Vissim.Net.Links:
            i.SetAttValue('LinkEvalSegLen', 99999)
    else:
        Vissim.Net.Links.ItemByKey(LinkNum).SetAttValue('LinkEvalSegLen', 99999)
        
def set_linkEvalSeg_len (LinkNum, length): ## Set link evaluation segment as 99999m for getting a whole link density
    if LinkNum==0:
        for i in Vissim.Net.Links:
            i.SetAttValue('LinkEvalSegLen', length)
    else:
        Vissim.Net.Links.ItemByKey(LinkNum).SetAttValue('LinkEvalSegLen', length)

        
def set_externalDLL(VehType, DLLFile): ## Set external DLL file, Veh type should be in list type, DLL type is integer, 0 means no DLL, 1 to 3 means conservative to aggressive driving respectively

    if DLLFile==0:
        for i in VehType:
            Vissim.Net.VehicleTypes.ItemByKey(i).SetAttValue('ExtDriver', False)
    else:
        for i in VehType:
            Vissim.Net.VehicleTypes.ItemByKey(i).SetAttValue('ExtDriver', True)
            DLLpath = os.path.join(DLLfolder, DLLFile)
            Vissim.Net.VehicleTypes.ItemByKey(i).SetAttValue('ExtDriverDLLfile', DLLpath)
    return (0)


# Generate Scenario (Only run 1 time)

In [ ]:


########################################################
################### Create Scenarios ###################
########################################################
## Run only first time to create scenarios!!!!!!!!!!!!!!
## Run only first time to create scenarios!!!!!!!!!!!!!!
## Run only first time to create scenarios!!!!!!!!!!!!!!
## Run only first time to create scenarios!!!!!!!!!!!!!!
## Run only first time to create scenarios!!!!!!!!!!!!!!


## Set Scenario file directory and file name

Scenario_dir = Path_of_COM_Basic_Commands_network + '\\Scenarios'
Scenario_file = os.path.join(Scenario_dir, 'Market Penetration_I-95.csv')

## Get Scenario file and save it to MP_Scenario
MP_Scenario = get_vehicle_composition_scenario(Scenario_file)


## Load file, make a list of scenario
MP_Scenario_List = MP_Scenario['MP Scenario'].tolist()
## Generate Vehicle Composition Scenario by MP_Scenario List(Only includes the ID number of scenario)


SpdDist_PV = [1053, 1053, 1053, 1053] ## Distribution for Intermediate, aggressive, conservative, manufacturer liability order
SpdDist_LT = [1055, 1055, 1055, 1055] ## Distribution for Intermediate, aggressive, conservative, manufacturer liability order
SpdDist_HT = [1057, 1057, 1057, 1057] ## Distribution for Intermediate, aggressive, conservative, manufacturer liability order

for n in range(len(SpdDist_PV)):
    gen_vehicle_composition(MP_Scenario_List, SpdDist_PV[n], SpdDist_LT[n], SpdDist_HT[n]) 
    set_vehicle_composition_scenario(MP_Scenario)

# DLLfolder = Path_of_COM_Basic_Commands_network + '\\DLL V6.2'
# DLL = ['DriverModel_I.dll', 'DriverModel_A.dll', 'DriverModel_C.dll', 'DriverModel_L.dll']


# for n in range(len(DLL)):
#     gen_vehicle_composition(MP_Scenario_List, SpdDist_PV[n], SpdDist_LT[n], SpdDist_HT[n]) 

#     print (MP_Scenario_List, SpdDist_PV, SpdDist_LT, SpdDist_HT)

#     ## Set Market Penetration Scenario by MP_Scenario (Includes percentage of each vehicle)
#     set_vehicle_composition_scenario(MP_Scenario)

#     ## Before run the simulation, set link evaluation segment length as 100ft for getting a speed, density heatmap
#     ##set_linkEvalSeg_99999(0)
#     ##set_linkEvalSeg_len (0, 100)

#     ## Set external DLL file, please check the path in set_externalDLL file function
#     ## Vehicle type should be a list
#     ## Second term 0: No DLL file, 1: Conservative scenarion DLL, 2: Moderate scenario DLL, 3: Aggressive scenario DLL

#     ## Set DLL file for the Vehicles

#     DLLfile = os.path.join(DLLfolder, DLL[n])
#     DLL_Veh_Type=[800, 810, 820, 900, 910, 920, 950, 960 ,970]
#     set_externalDLL(DLL_Veh_Type, DLLfile)


print ('Generate all scenarios and load all DLL files')


# Output Function

In [4]:
def Output (Randseed, File, TTM, DCM, Density):
    
    ## Output Directory and file name
    Output_Dir = Path_of_COM_Basic_Commands_network + '\\Outputs'
    TTM_Output_file = os.path.join(Output_Dir, Randseed + File + '_I95_MOE_TTM_Output.csv')
    DCM_Output_file = os.path.join(Output_Dir, Randseed + File + '_I95_MOE_DCM_Output.csv')
    Density_Output_file = os.path.join(Output_Dir, Randseed + File + '_I95_MOE_Density_Output.csv')
    # Delay_Output_file = os.path.join(Output_Dir, Scenario + 'I95_MOE_Delay_Output.csv')


    ## Set pandas dataframe
    MOE_TTM_df = pd.DataFrame(Output_TTM)
    MOE_TTM_df.columns = ['RandomSeed', 'Scenario', 'TTM_No', 'TTM_Name', 'Time Interval', 'Travel Time', 'Number of Vehicle']

    MOE_DCM_df = pd.DataFrame(Output_DCM)
    MOE_DCM_df.columns = ['RandomSeed', 'Scenario', 'DCM_No', 'Time Interval', 'No_Veh', 'Speed', 'Acc', 'len', 'Delay']
    
    MOE_Density_df = pd.DataFrame(Output_Density)
    MOE_Density_df.columns = ['RandomSeed', 'Scenario', 'Link_No', 'Time Interval',  'Segnemt_No', 'Density', 'Speed', 'Volume']
    # MOE_Delay_df = pd.DataFrame(Output_Delay)
    # MOE_Delay_df.columns = ['RandomSeed', 'Scenario', 'TTM_No', 'Time Interval', 'VehDelay', 'Number of Vehicle']


    MOE_TTM_df.to_csv(TTM_Output_file)
    MOE_DCM_df.to_csv(DCM_Output_file)
    MOE_Density_df.to_csv(Density_Output_file)
    # MOE_Delay_df.to_csv(Delay_Output_file)

    # print (MOE_TTM_df)
    # print (MOE_DCM_df)
    # print (MOE_QC_df)
    # print (MOE_Density_df)
    # print (MOE_Delay_df)

# Run Simulation

In [ ]:
## Define Simulation parameters
Randomseed=[5, 10, 15, 20, 45] #, 30 ,37, 40 ,45]
##Randomseed=list(range(1,101))
SimEndTime = 9000
##MP_Scenario_List = [100, 115, 120]
##MP_Scenario_List = [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 116, 117, 118, 119]


## Set Time Interval for result
NumOfInterval = 10
TimeInterval=list(range(1,NumOfInterval+1))
print ("Time Interval for Result:", TimeInterval)
## Set Time Interval for result

## Set Time Interval for input
NumOfInputInterval = 14
TimeInterval_number=list(range(1,NumOfInputInterval+1))
print ("Time Interval for Input:", TimeInterval_number)

########################################################
## Simulation 
########################################################

##veh_input_list = [3000, 3500, 4000, 4500, 5000, 5500, 6000]
## Run Simulations
#for i in range(N_of_scenario):
for randseed in Randomseed:
    # Change Randomseed and run simulationi
    # Number of simulation = Number of random seed * Number of vehicle composition list * number of vehicle input change...
    ##for veh_input in veh_input_list:
    Output_DCM = []
    Output_TTM = []
    Output_QC = []
    Output_Density = []
    Output_Delay = []

    start_time = time.time()
    # Vehicle Compostion change. If you want to change all vehicle input to same composition, put 0 in vehicle input number
    # Structure: vehicle_compositioni_change(vehicle input number, vehicle composition number)
    ##set_vehicle_composition(0, TimeInterval_number, veh_comp)
    ##set_vehicle_input(0, veh_input)

    # Run Simulation with randseed
    print ('Simulation starts with randomseed %s \n' %(randseed))
    simulation(randseed, SimEndTime)


    # Get Results, Input 0 means all
    MOE_TTM = get_traveltime(0, TimeInterval) # MOE from travel time measurement, Return list [ID, TTM_Name, TimeInterval, Travel Time, Number of Vehicles]
    MOE_DCM = get_data_DCM(0, TimeInterval) # MOE from Data Collection measurement, Return list [[DCM_No, TimeInterval, No_Veh, Speed, Acceleration, Length, Delay]] 
    ##MOE_QC = get_queue(0, TimeInterval) # MOE from Queue Counter, Return list [[QC_No, TimeInterval, Max Queue]] 
    MOE_Density = get_density(0, TimeInterval) # MOE from link, Return list[[Link_No, TimeInterval, Density]]


    end_time = time.time()
    Sim_period = end_time - start_time
    print ('%.2f seconds for single run \n' % (Sim_period))

    ## Output MOE_TTM
    for i in MOE_TTM:
        Tmp_TTM = [randseed, File, i[0], i[1], i[2], i[3], i[4]] ## Randomseed, VehicleComposition, TTM_ID, TTM_Name, TimeInterval, Travel Time, Number of Vehicles
        Output_TTM.append(Tmp_TTM)
    ## Output MOE_DCM
    for i in MOE_DCM:
        Tmp_DCM = [randseed, File, i[0], i[1], i[2], i[3], i[4], i[5], i[6]] ## Randomseed, VehicleComposition, DCM NO, TimeInterval, #ofVeh, Spd, Acc, Len, Delay
        Output_DCM.append(Tmp_DCM)

    ## Output MOE_Density
    for i in MOE_Density:
        Tmp_Density = [randseed, File, i[0], i[1], i[2], i[3], i[4], i[5]] ## Link_No, TimeInterval, Density, Speed, Volume
        Output_Density.append(Tmp_Density)



    Output(str(randseed), File, Output_TTM, Output_DCM, Output_Density)



Time Interval for Result: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Time Interval for Input: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Simulation starts with randomseed 5 

Start at 2020-10-10 12:30:58 | with Randomseed 5 | Simulation ends at 9000
36258.51 seconds for single run 

Simulation starts with randomseed 10 

Start at 2020-10-10 22:35:17 | with Randomseed 10 | Simulation ends at 9000
36146.42 seconds for single run 

Simulation starts with randomseed 15 



In [16]:
Output(str(randseed), str(veh_comp), Output_TTM, Output_DCM, Output_QC, Output_Density)

In [9]:
    for i in MOE_DCM:
        Tmp_DCM = [randseed, File, i[0], i[1], i[2], i[3], i[4], i[5], i[6]] ## Randomseed, VehicleComposition, DCM NO, TimeInterval, #ofVeh, Spd, Acc, Len, Delay
        Output_DCM.append(Tmp_DCM)

    ## Output MOE_Density
    for i in MOE_Density:
        Tmp_Density = [randseed, File, i[0], i[1], i[2], i[3], i[4], i[5]] ## Link_No, TimeInterval, Density, Speed, Volume
        Output_Density.append(Tmp_Density)

In [10]:
Output(str(randseed), File, Output_TTM, Output_DCM, Output_Density)